In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed

In [3]:
addprocs(1)

1-element Vector{Int64}:
 2

In [4]:
@everywhere using DistributedStreams

Precompiling DistributedStreams
  ✓ Chain
  ✓ DistributedStreams
  2 dependencies successfully precompiled in 2 seconds. 18 already precompiled.


In [5]:
input, output, control = launch_monitor(
    x->begin
        id = x.id
        data = DistributedStreams.Serialize.deserialize(x.data) + 1
        Entry(id=x.id, data=DistributedStreams.Serialize.serialize(data), valid=true)
    end;
    start_safe = false,
    verbose = true
)

(RemoteChannel{Channel{Entry}}(1, 1, 12), RemoteChannel{Channel{Entry}}(1, 1, 13), DistributedStreams.RemoteWorkerControl(@NamedTuple{worker::Int64, safe::Base.RefValue{Bool}, flag::Base.RefValue{Bool}}[(worker = 2, safe = Base.RefValue{Bool}(false), flag = Base.RefValue{Bool}(false)), (worker = 3, safe = Base.RefValue{Bool}(false), flag = Base.RefValue{Bool}(false))], Any[Future(2, 1, 20, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139839803437712, 139839775257376)), nothing), Future(3, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 139839834038624, 3)), nothing)]))

In [6]:
control.status

2-element Vector{Any}:
 Future(2, 1, 20, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139839803437712, 139839775257376)), nothing)
 Future(3, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 139839834038624, 3)), nothing)

In [7]:
put!(input, Entry(id=0, data=DistributedStreams.Serialize.serialize("hi there"), valid=false))

RemoteChannel{Channel{Entry}}(1, 1, 12)

In [8]:
typeof(control.status[1])

Future

In [17]:
t = @async fetch(control.status[2])

Task (runnable) @0x00007f2ebbbf6400

In [18]:
istaskdone(t)

true

In [19]:
t.result

RemoteException(3, CapturedException(MethodError(+, ("hi there", 1), 0x0000000000007aea), Any[(#1 at In[5]:4, 1), (macro expansion at task.jl:479 [inlined], 1), (remote_worker at DistributedStreams.jl:156, 1), (#invokelatest#2 at essentials.jl:892, 1), (invokelatest at essentials.jl:889, 1), (#107 at process_messages.jl:282, 1), (run_work_thunk at process_messages.jl:70, 1), (run_work_thunk at process_messages.jl:79, 1), (#100 at process_messages.jl:88, 1)]))

In [20]:
typeof(t.result)

RemoteException

In [21]:
typeof(fetch(t))

LoadError: TaskFailedException

[91m    nested task error: [39mOn worker 3:
    MethodError: no method matching +(::String, ::Int64)
    
    [0mClosest candidates are:
    [0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
    [0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m
    [0m  +([91m::BigFloat[39m, ::Union{Int16, Int32, Int64, Int8})
    [0m[90m   @[39m [90mBase[39m [90m[4mmpfr.jl:447[24m[39m
    [0m  +([91m::Base.CoreLogging.LogLevel[39m, ::Integer)
    [0m[90m   @[39m [90mBase[39m [90m[4mlogging.jl:131[24m[39m
    [0m  ...
    
    Stacktrace:
     [1] [0m[1m#1[22m
    [90m   @[39m [90m./[39m[90m[4mIn[5]:4[24m[39m
     [2] [0m[1mmacro expansion[22m
    [90m   @[39m [90m./[39m[90m[4mtask.jl:479[24m[39m[90m [inlined][39m
     [3] [0m[1mremote_worker[22m
    [90m   @[39m [90m~/Developer/DistributedStreams/src/[39m[90m[4mDistributedStreams.jl:156[24m[39m
     [4] [0m[1m#invokelatest#2[2

In [16]:
collect!(output)

Entry[]